In [ ]:
import numpy as np
import h5py
import datetime
import os
from datetime import datetime
import matplotlib.pyplot as plt
from os import listdir
from matplotlib.mlab import psd

from scipy import integrate, optimize
from scipy.stats import chi2

def every_nth(nums, nth):
    # Use list slicing to return elements starting from the (nth-1) index, with a step of 'nth'.
    return nums[nth - 1::nth]

def calibrate_frequency_domain(freq_domain, freqs, calibration_factors, phase_data):
    calibrated_freq_domain = np.zeros_like(freq_domain, dtype=complex)
    for i, freq in enumerate(freqs):
        if freq >= 0:  # Only process positive frequencies
            calibration_factor = calibration_factors[i]
            phase = phase_data[i]

            # Apply calibration factor and phase correction
            # the 2* is for one sided freq>0
            calibrated_freq_domain[i] = 2 * freq_domain[i] / calibration_factor * np.exp(1j * phase)
    return calibrated_freq_domain

def compute_noise_spectral_density(time_series, sampling_rate):
    # Calculate the length of the time series
    n = len(time_series)

    # Compute the Fast Fourier Transform (FFT)
    fft_result = np.fft.fft(time_series)

    # Calculate the one-sided power spectral density
    psd = (1 / (sampling_rate * n)) * np.abs(fft_result[:n//2])**2

    # Calculate the corresponding frequencies
    freqs = np.fft.fftfreq(n, 1/sampling_rate)[:n//2]

    return freqs, np.sqrt(psd)

# seconds from (1970/01/01 00:00:00.0) to gps epoch (1980/01/06 00:00:19.0), ignoring leap-seconds
gpsEpoch = 315964819.
# Constants
SampleRate = 2597
rho = 6.04e7 # in nT^2 dark matter density in magnetic field units
R = 0.0212751 # in Hz^-1 Radius of earth divided by c
fd = 1 / 86164 # in Hz, rotation frequency of the Earth (1/day)
dT = 1/SampleRate # sampling period (in s)

In [ ]:
pathLemmy = '/Users/gc2138/Desktop/SNIPE/LemmySimul'
pathPhil = '/Users/gc2138/Desktop/SNIPE/PhilSimul'

os.chdir(pathLemmy)
folder_files = os.listdir(pathLemmy)

#Load Time Series from File
# put data into a list
BdataN = [];
concatTimeN = [];
BdataE = [];
concatTimeE = [];

downsampleFactor = 1; # originally had because Katie's computer couldn't handle large amounts of data
scaleFactor = 1; # to account for amplification after LEMI

for file in folder_files:
    with h5py.File(file, 'r') as f:
        dataN = f['data'][()] * scaleFactor
        timestamps = f['timestamps'][()]
        concatTimeN.extend(timestamps)

        #Downsampling data
        for n in range(len(dataN[0,:])):
              nums = every_nth(dataN[:,n], downsampleFactor)
              BdataN.extend(nums)

os.chdir(pathPhil)
folder_files = os.listdir(pathPhil)

for file in folder_files:
    with h5py.File(file, 'r') as f:
        dataE = f['data'][()] * scaleFactor
        timestamps = f['timestamps'][()]
        concatTimeE.extend(timestamps)

        #Downsampling data
        for n in range(len(dataE[0,:])):
              nums = every_nth(dataE[:,n], downsampleFactor)
              BdataE.extend(nums)

In [ ]:
# load lemmy and phil data, get coords for our station, calibrate data after FFT

#convert coordinates into radians
[latL,longL] = (39 + 6.024/60)*np.pi/180, (120 + 55.386/60)*np.pi/180
[latP, longP] = (39 + 6.101/60)*np.pi/180, (120 + 55.426/60)*np.pi/180

#convert to spherical coordinates for theta
latL = np.pi/2 - latL;
LatP = np.pi/2 - latP;

lat1 = latL
long1 = longP

#Range of frequencies analyzed
lowFreq,highFreq = [0.1,5.0];

N = len(BdataN)
fp = int(lowFreq*N*dT) # this picks out the data point in our list corresponding to lowFreq = Ntot LowFreq/(frequency span)
fq = int(highFreq*N*dT)+1;


# Construct data vector
fdhat = int(np.round(fd * N * dT)) # fdhat index. fdhat is closest discrete frequency interval to fd (fd = 1/day)
FFT1 = -dT*np.fft.fft(BdataN)
FFT2 = dT*np.fft.fft(BdataE)

os.chdir('/Users/gc2138/Desktop/SNIPE')
calibrationL = 'LEMMYCalibration691text.csv'
calibrationP = 'PHILCalibration748text.csv'

freqsN, noise_spectral_densityN = compute_noise_spectral_density(BdataN, SampleRate)
freqsE, noise_spectral_densityE = compute_noise_spectral_density(BdataE, SampleRate)

calibration_dataL = np.loadtxt(calibrationL, delimiter = ",")
calibration_dataP = np.loadtxt(calibrationP, delimiter = ",")

frequency_calibrationL = calibration_dataL[:, 0]  # Frequency values in Hz
voltage_calibrationL = 10**-3 * calibration_dataL[:, 1]    # Volts per nano-Tesla (original calibration data is in mV/nT)
phase_calibrationL = calibration_dataL[:,2]

frequency_calibrationP = calibration_dataP[:, 0]  # Frequency values in Hz
voltage_calibrationP = 10**-3 * calibration_dataP[:, 1]    # Volts per nano-Tesla (original calibration data is in mV/nT)
phase_calibrationP = calibration_dataP[:,2]

# calculates the calibration factor at every frequency in FFT of our data
voltage_calibration_interpolatedN = np.interp(np.abs(freqsN), frequency_calibrationL, voltage_calibrationL)
phase_correction_interpolatedN = np.interp(np.abs(freqsN), frequency_calibrationL, phase_calibrationL)
voltage_calibration_interpolatedE = np.interp(np.abs(freqsE), frequency_calibrationP, voltage_calibrationP)
phase_correction_interpolatedE = np.interp(np.abs(freqsE), frequency_calibrationP, phase_calibrationP)

calFFT1 = calibrate_frequency_domain(FFT1, freqsN, voltage_calibration_interpolatedN, phase_correction_interpolatedN)
calFFT2 = calibrate_frequency_domain(FFT2, freqsE, voltage_calibration_interpolatedE, phase_correction_interpolatedE)

In [ ]:
# Vector of complete FT data at Compton frequency + sidebands
X = np.stack([calFFT1[fp - fdhat : fq - fdhat], calFFT2[fp - fdhat : fq - fdhat],
              calFFT1[fp : fq], calFFT2[fp : fq],
              calFFT1[fp + fdhat : fq + fdhat], calFFT2[fp + fdhat : fq + fdhat]])

# Compute expectation value
# Discrete sampling correction for sidebands
Q = lambda f: (1 - np.exp(-2 * np.pi * 1j * f * N * dT)) / (1 - np.exp(-2 * np.pi * 1j * f * dT))

# basis vectors for dark matter signal
mu0 = -N * dT * np.sqrt(rho / 2) * np.array([
    0, 0, 0, np.sin(lat1), 0, 0])
muplus = -dT * np.sqrt(rho) / 2 * np.array([
    1j * np.exp(-1j * long1) * Q(fd - fdhat / N / dT), np.cos(lat1) * np.exp(-1j * long1) * Q(fd - fdhat / N / dT),
    1j * np.exp(-1j * long1) * Q(fd), np.cos(lat1) * np.exp(-1j * long1) * Q(fd),
    1j * np.exp(-1j * long1) * Q(fd + fdhat / N / dT), np.cos(lat1) * np.exp(-1j * long1) * Q(fd + fdhat / N / dT)])
muminus = -dT * np.sqrt(rho) / 2 * np.array([
    -1j * np.exp(1j * long1) * Q(fd - fdhat / N / dT), np.cos(lat1) * np.exp(1j * long1) * Q(fd - fdhat / N / dT),
    -1j * np.exp(1j * long1) * Q(fd), np.cos(lat1) * np.exp(1j * long1) * Q(fd),
    -1j * np.exp(1j * long1) * Q(fd + fdhat / N / dT), np.cos(lat1) * np.exp(1j * long1) * Q(fd + fdhat / N / dT)])
# Note that mu, nu, and S are all missing a factor of frequency, so that they can be frequency independent.  This factor will be added back when computing the posterior.

In [ ]:
# Compute covariance matrix
Sigma = np.sum(X[2:4, None] * np.conj(X[2:4]), axis = -1) / (fq - fp)

inva = np.linalg.inv(np.linalg.cholesky(Sigma))

invA = np.block([[inva, np.zeros((2, 2)), np.zeros((2, 2))],
                 [np.zeros((2, 2)), inva, np.zeros((2, 2))],
                 [np.zeros((2, 2)), np.zeros((2, 2)), inva]])

In [ ]:
# Compute S and Z
Y = np.matmul(invA, X)
nu0 = np.matmul(invA, mu0)
nuplus = np.matmul(invA, muplus)
numinus = np.matmul(invA, muminus)
U, S, Vh = np.linalg.svd(np.stack([nuplus, nu0, numinus]).T, full_matrices = False)
Z = np.matmul(np.conj(U.T), Y)

In [ ]:
# Compute posterior and determine bound

## Compute Limits
import warnings
#warnings.filterwarnings("ignore")
warnings.resetwarnings();
warnings.simplefilter('always');

# Compute posterior and determine bound
bound = []
print("Calculating Bounds")
try:
    for n in range(fq - fp):
        mR = 2 * np.pi * (fp + n) / N / dT * R
        Sn = S * mR / (2 - (mR) ** 2) # re-introduce frequency factor
        Zn = Z[:, n]
        pdf = lambda eps: np.sqrt(np.sum(4 * eps ** 2 * Sn ** 4 / (3 + eps ** 2 * Sn ** 2) ** 2)) * np.prod(np.exp(- 3 * np.abs(Zn) ** 2 / (3 + eps ** 2 * Sn ** 2)) / (3 + eps ** 2 * Sn ** 2))
        upper = 10 / min(Sn)
        norm = integrate.quad(pdf, 0, upper)[0]
        while integrate.quad(pdf, 0, 2 * upper)[0] > norm * 1.01:
            print(n)
            upper *= 2
            norm = integrate.quad(pdf, 0, upper)[0]
        normedpdf = lambda eps: pdf(eps) / norm
        epp_guess = 1 / max(Sn)
        limit = optimize.fsolve(lambda eps: integrate.quad(normedpdf, 0, eps)[0] - 0.95, epp_guess)[0]
        bound.append(limit)
        print("n = %2d, bound = %2.6f" %(n,limit))
except RuntimeWarning:
    breakpoint()

In [ ]:
fname='SNIPE23Scan2024_04_18'+str(lowFreq)+'.hdf5';
hf = h5py.File(fname, 'w')
hf['frange']=[0.1,5]
hf['Z'] = Z;
hf['bounds'] = bound
hf.close()